In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

Matplotlib is building the font cache; this may take a moment.


In [3]:
def f(x):
    return 3*x**2 - 4*x + 5

In [4]:
f(3.0)

20.0

In [5]:
xs = np.arange(-5,5,0.25)
ys = f(xs)
plt.plot(xs, ys)

In [6]:
#derivative
#함수의 특정 지점 a에서 작은 수인 h를 더하면 함수는 어떻게 반응할 것인가 어떤 민감도로. 그때의 경사는 어떻게, 얼마나 올라가는지 내려가는지
h = 0.00001 #너무 작은 값을 사용하면 메모리의 한계로 오류
x = 3.0
f(x + h) #3.0지점에서 양의 방향으로 살짝 가면 원래의 20보다 살짝 높은 값일것임
(f(x + h) - f(x)) / h #함수가 얼마나 반응 했는지를 normalize -> 기울기

14.00003000000538

In [7]:
a = 2.0
b = -3.0
c = 10.0
d = a*b +c
print(d)

4.0


In [8]:
#derivative of a function with multiple inputs
h = 0.0001

#inputs
a = 2.0
b = -3.0
c = 10.0

d1 = a*b +c
#기울기가 a에 대해서 어떻게 영향을 받나

a+=h
d2 = a*b + c

print ('d1',d1)
print ('d2',d2)
print ('slope',(d2-d1)/h) # d2-d1 => 우리가 특정 입력들을 증가시켰을때 함수가 얼마나 증가했나, 
                          # h로 나누어 normalize
                          # d1은 4.0이고 a가 살짝 커지고 나머지는 동일하다면 결과는 더 작아질 것으로 예상
                          # 기울기도 음수일 것으로 예상
                          # b에 h를 더한다면 오히려 값이 증가, 기울기가 양수
                          # c에 h를 더한다면 값은 딱 h만큼 증가 - > 기울기  1

d1 4.0
d2 3.999699999999999
slope -3.000000000010772


In [9]:
 class Value:

    def __init__(self, data, _children=(), _op='', label = ''):
        self.data = data
        self.grad = 0.0 # gradient, 초기 값에는 value가 결과에 영향을 끼치지 않도록 기울기를 0
        self._backward = lambda : None #chain rule을 하는 함수. output grad을 input grad에 chain. 초기에는 아무것도 안함
        self._prev = set(_children)
        self._op = _op
        self.label = label
        
    def __repr__(self): 
        return f"Value(data={self.data})"
    
    def __add__(self,other):
        other = other if isinstance(other, Value) else Value(other) #만약 other가 숫자라면 Value로 만들어줌. -> .data사용 
        out = Value(self.data + other.data, (self, other),'+')
        
        def _backward():
            #만약 self와 other가 같다면 1로 만들어주고 다시 1로 만들어주는 에러
            #self.grad = 1.0 * out.grad 
            #other.grad = 1.0 * out.grad
            self.grad += 1.0 * out.grad 
            other.grad += 1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other),'*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    def __pow__(self, other): #other는 반드시 int or float
        assert isinstance(other,(int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other,(self,), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other -1)) * out.grad # d/dx x^n = n * x^n-1
        out._backward = _backward
        return out
    
    def __rmul__(self, other): # other * self. 2 * a가 안되면 a * 2로 반호
        
        return self*other
    def __radd__(self, other): # other + self
        return self + other
    
    def __truediv__(self, other): # self / other
        return self * other**-1
    
    def __neg__(self): # -self
        return self * -1
    
    def __sub__(self, other): # self - other
        return self + (-other)
    
    def tanh(self): #tanh
        x = self.data
        t = (math.exp(2*x)-1) / (math.exp(2*x) + 1)
        out = Value(t,(self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad #tanh의 미분은 1 - t**2
        
        out._backward = _backward
        
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x),(self, ), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad 
        out._backward = _backward
        
        return out
        
        
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev: 
                    build_topo(child)
                topo.append(v) 
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
                 
a = Value(2.0,label='a') #__repr_함수로 문장 출력
b = Value(-3.0,label='b')
# a + b          #__add__함수 없으면 실행 불가
c = Value(10.0,label='c')
#(a.__mul__(b)).__add__(c)
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0,label='f')
L = d * f; L.label = 'L'
L
#서로 연결이 필요, 포인터
#Value를 생성하면 children에게 self, other를 먹임
#_op은 연산자 add에서는 +, mul에서는 * ...

Value(data=-8.0)

In [10]:
d._prev # 위에서 먹인 값들을 _prev (children)에

{Value(data=-6.0), Value(data=10.0)}

In [11]:
d._op #d는 a*b와 c의 add로 만들어졌다

'+'

In [12]:

from graphviz import Digraph

def trace(root):
    #build a set of all node and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) #, node_attr={'rankdir': 'TB'})
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name=uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            # if this value is a result of some opreation, create an op node for it
            dot.node(name=str(id(n)) + n._op, label=n._op)
            #and connect this node to it
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

# 그래프를 그려줌 연산자가 그려진 노드는 실제 노드는 아니다. 사각형이 실제 데이터를 가진 것

ModuleNotFoundError: No module named 'graphviz'

In [ ]:
draw_dot(L) # L을 만든 수학적 표현을 보여줌. L부터 시작해서 거꾸로.
            # L의 f에 대한 미분, e에대한 ,c에 대한 ... -> 신경망에서는 가중치에 대한 loss function L의 미분. 각노드의 데이터에 대한게아니라
            # 미분(기울기)을 저장할 변수 grad 
        

: 

In [ ]:
L = d * f

# dl/dd= ? f

(f(x+h)-f(x))/h
=((d+h)*f - d*f)/h
=(d*f*h*f - d*f)/h
=(h*f)/h
=f

: 

In [13]:
f.grad=4.0
L.grad = 1
d.grad=-2.0
c.grad =-2.0
e.grad=-2.0
a.grad = (-2.0 * -3.0)
b.grad = -2.0 * 2.0

In [63]:
a.data += 0.01 * a.grad
b.data += 0.01 * b.grad
c.data += 0.01 * c.grad
f.data += 0.01 * f.grad

e = a * b
d = e + c
L = d * f

print(L.data)
#전 노드의 값을 변경시키면 L에도 영향

-1.9761969838327742e-14


In [65]:
def lol():
    h=0.001
    
    a = Value(2.0,label='a') 
    b = Value(-3.0,label='b')
    c = Value(10.0,label='c')
    e = a*b; e.label = 'e'
    d = e + c; d.lable = 'd'
    f = Value(-2.0,label='f')
    L = d * f; L.label = 'L'
    L1 = L.data
    
    a = Value(2.0,label='a') 
    #a.data+=h
    b = Value(-3.0,label='b')
    b.data+=h
    c = Value(10.0,label='c')
    #c.data+=h
    e = a*b; e.label = 'e'
    #e.data+=h
    d = e + c; d.lable = 'd'
    #d.data+=h
    f = Value(-2.0,label='f')
    L = d * f; L.label = 'L'
    L2 = L.data
    
    print((L2 - L1)/h)

lol()

-3.9999999999995595


In [ ]:
dL/dc
how is L sensitive to c (through d)?
=> Chain Rule
As put by George F. Simmons: 
    "If a car travels twice as fast as a bicycle and the bicycle is four times as fast as a walking man, 
     then the car travels 2 × 4 = 8 times as fast as the man."

WANT:        
dL / dc = (dL / dd) * (dd / dc)

KNOW:
dL / dd
dd / dc


dL /de = -2.0

e = a*b
de / da? b

dL / da = (dL / de) * (de / da) L에서부터 chain rule을 통해 a에 대한 미분값도 구할 수 있음 (L에 대한 a의 영향)



dd / dc ? 1.0
d = c + e

dd / de ? 1.0



: 

In [ ]:
#Backpropagation neuron

plt.plot(np.arange(-5,5,0.2), np.tanh(np.arange(-5,5,0.2))); plt.grid();

: 

In [ ]:
#inputs x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
#weight w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
#bias of the neuron
b = Value(6.8813733429827587, label='b')
#x1w1 + x2w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1*w1 + x2*w2'
n =x1w1x2w2 + b; n.label = 'n'

o = n.tanh(); o.label = 'o'



: 

In [ ]:
draw_dot(o)
# w2는 기울기가 없어 영향이 없음.
# w1은 양의 기울기 뉴런의 결과도 상승

: 

: 

In [ ]:
o.backward()

: 

In [ ]:
o.grad = 1.0

topo = []
visited = set()
def build_topo(v):
    if v not in visited:
        visited.add(v)
        for child in v._prev: 
            build_topo(child)
        topo.append(v) 
build_topo(o)

for node in reversed(topo):
    node._backward()


: 

In [ ]:
# _backward를 각각 부르는 것도 제거
#
topo = []
visited = set() #visited nodes
def build_topo(v):
    if v not in visited:
        visited.add(v)
        for child in v._prev: #children도 topo
            build_topo(child)
        topo.append(v) #모든 자식들이 끝나야 추가
build_topo(o)
topo #마지막이 o 그 뒤로 n, ...순으로 정렬

: 

In [ ]:
o.grad = 1.0 #grad가 0이었으르모 _backward를 부르려면 설정해줘야함

: 

In [ ]:
o._backward() #n의 grad를 정함

: 

In [ ]:
n._backward()

: 

In [ ]:
b._backward() #b는 뒤가 없으므로 아무것도 하지 않는 함수로 불려짐

: 

In [ ]:
x1w1x2w2._backward()

: 

In [ ]:
x1w1._backward()

: 

In [ ]:
x2w2._backward()

: 

In [ ]:
#backproapagation manually
x1.grad = w1.data * x1w1.grad
w1.grad = x1.data * x1w1.grad

: 

In [ ]:
x2.grad = w2.data * x2w2.grad
w2.grad = x2.data * x2w2.grad

: 

In [ ]:
x1w1.grad = 0.5
x2w2.grad = 0.5

: 

In [ ]:
x1w1x2w2.grad = 0.5
b.grad = 0.5

: 

In [ ]:
n.grad = 0.5

: 

In [ ]:
o.grad = 1.0

: 

In [ ]:
o.data

: 

In [ ]:
1 - o.data**2

: 

In [ ]:
# o = tanh(n)
# do / dn = 1-tanh(n)**2 = 1 - o**2

: 

In [ ]:
#implementing the backward function for eaxh operation


: 

In [ ]:
#error case 1
a = Value(3.0, label='a')
b = a + a;    b.label = 'b'
b.backward()
draw_dot(b)

# derivative of b respct to a는 2가 되야 하지만 1이라고 표시

: 

In [ ]:
# error 2
a = Value(-2.0, label='a')
b = Value(3.0, label='b')
d = a * b; d.label = 'd'
e = a + b; e.label = 'e'
f = d * e; f.label = 'f'

f.backward()

draw_dot(f)

# 한번 이상 사용되면 에러
# grad를 더해줌

: 

In [ ]:
a = Value(2.0)
a + 1 # 숫자 1에는 .data가 없음

: 

In [ ]:
a*2

: 

In [ ]:
2*a

: 

In [ ]:
a.exp()

: 

In [ ]:
a = Value(2.0)
b = Value(4.0)
a / b

a * (b**-1)


: 

In [ ]:
#inputs x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
#weight w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
#nias of the neuron
b = Value(6.8813733429827587, label='b')
#x1w1 + x2w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1*w1 + x2*w2'
n =x1w1x2w2 + b; n.label = 'n'

# o = n.tanh(); o.label = 'o'
#---- tanh를 분해
e = (2*n).exp()
o = (e - 1) / (e + 1)
#-----
o.label = 'o'
o.backward()
draw_dot(o)

: 

In [ ]:
# 위의 동작을 파이토치를 이용해서
# tensor : n-dimensional arrays of scalars
import torch 

: 

In [ ]:


# 2.0스칼라 값을 가지는 텐서를 만들고 double로 변환(파이썬은 double은 사용해서),
# leaf node라서 default로는 gradient가 필요하지 않다생각 -> .requires_grad를 True로
x1 = torch.Tensor([2.0]).double()                ; x1.requires_grad = True 
x2 = torch.Tensor([0.0]).double()                ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()               ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()                ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double()  ; b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

#.item()으로 값 가져오기
print(o.data.item())
o.backward()

print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

: 

In [ ]:
o.item()

: 

In [ ]:
#Building out a neural net library (multi percoptron)
class Neuron:
    def __init__(self,nin): #nin = number of input
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
    
    def __call__(self,x):
        # x = [2.0 3.0]
        # n = Neuron(2)
        # n(x) -> call사용
        # zip = 두 데이터들을 같은 인덱스끼리 묶어줌 [1, 2] [a, b] => (1, a), (2, b)
        # w * x + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)) , self.b) 
        out = act.tanh()
        return out
    
    def parameters(self): # pitorch에도 있는 기능
        return self.w + [self.b] #w list + b list => list
    
#define layer of neuron
class Layer:
    
    def __init__(self, nin, nout): #nout - number of output 하나의 레이어에 얼마나 많은 뉴런?
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
        #위와 똑같은 동작
        #params = []
        #for neuron in self.neurons:
        #    ps = neuron.parameters()
        #    params.extend(ps) # params 끝에 집어넣음
        #return params
        
#MLP (multi layer percoeptron)
class MLP:

    def __init__(self, nin, nouts):  #
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

        
#x = [2.0, 3.0, -1.0]

#n = Neuron(2) #2-dimentional neuron
#n = Layer(2,3) #2-dimentional nueron x 3
#n = MLP(3, [4, 4, 1]) #3차원 입력 , 2레이어는 4개의 output, 하나는 1개의 output
#n(x)

: 

In [ ]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1]) #3차원 입력 , 2레이어는 4개의 output, 하나는 1개의 output
n(x)

: 

In [ ]:
n.parameters()

: 

In [ ]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] #desired target
#ypred = [n(x) for x in xs] #how do we make neural net 
                           #how do we tune the weights to btter predict desired target?
#ypred

# 그를 위해서 뉴럴 넷의 모든 performance를 측정하는 하나의 숫자를 계산해야함 => loss
# 차이가 크면 loss가 큼

: 

In [ ]:

for k in range(20):
    #forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
    
    #backward pass
    for p in n.parameters(): #grad를 0으로 초기화 안하면 한번 돌때마다 step size가 늘어나는 효과.
        p.grad = 0.0

    loss.backward()
    
    #update
    for p in n.parameters():
        p.data += -0.05 * p.grad
    
    print(k, loss.data)
    

: 

In [ ]:
ypred

: 

In [ ]:
[(yout - ygt)**2 for ygt, yout in zip(ys, ypred)] # ygt = y ground truth
# 차이를 위해서 빼주고 양수이어야하므로 제곱, 타겟이면 0, 아니면 다른 숫자

: 

In [ ]:
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred)) #모든 차이를 더해줌.
loss # 클수록 예상을 못함

: 

In [ ]:
#forward pass
ypred = [n(x) for x in xs]
loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
loss # loss 줄어듬

: 

In [ ]:
# backward pass
loss.backward()

: 

In [ ]:
n.layers[0].neurons[0].w[0].grad # 특정 레이어의 한 뉴런의 weight가 loss를 올라가거나 내려가게한다.

: 

In [ ]:
n.layers[0].neurons[0].w[0].data

: 

In [ ]:
# update
for p in n.parameters():
    p.data += -0.01 * p.grad # 기울기 방향으로 조금만 증가 시킴 
                             # 기울기가 음수이고 값이 양수인 경우 값을 증가시키면 loss가 작아짐. 
# forward pass - backward pass - update 반복 loss 작아질때까지

: 

In [ ]:
n.layers[0].neurons[0].w[0].data

: 

In [ ]:
ypred

: 

In [ ]:
n.parameters()

: 

In [ ]:
what ar neural nets ?
-> mathmatical expression that take input as the data and they take input
-> mathmatical expression for the forward pass followed by loss function

loss 
-> 잘 예측했는지
-> loss를 backward해서 backpropagation을 이용해서 기울기를 알아내고 loss를 줄이기 위해 parameters를 조정
-> 이를 반복해서 로스를 최소화 그러면 net은 원하는 것을 할 수 있다
ex) gpt 인터넷 방대한 양의 텍스트가지고 뉴럴넷이 약간의 텍스트를 가지고 다음 테긋트를 예상하려한다


: 